## 1. Imports

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf


print(tf.__version__)


2.15.0


## 2. Load Dataset

### 2.2 Pad

### 2.3 Resize

In [7]:
class_names = [ "marigold", "jasmine", "rose", "hibiscus", "sunflower", "dahlia", "lotus", "bougainvillea", 
								"chrysanthemum", "lily", "lavender", "aloe-vera", "snake-plant", "golden-barrel-cactus"]
class_count = len(class_names)
print(class_count)

14


In [ ]:
# Load the files from dir to memory
# Resize Image to [224, 224, 3]
# Create a Train, Test, Validate Split
train_dataset
test_dataset
validate_dataset

## 7. Build Model

In [12]:
from tensorflow.keras import Model, Sequential, layers, applications
import tensorflow_hub as hub

def build_model():
    dimensions = 224
    # Input layer
    input_layer = layers.Input(shape=(dimensions, dimensions, 3))
    # Preprocessing the input
    preprocessing_layer = layers.Lambda(lambda x: applications.mobilenet_v3.preprocess_input(x))(input_layer)
    feature_extractor = hub.KerasLayer('https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5', trainable=False)(preprocessing_layer)
    hidden_layer = layers.Dense(512, activation='relu')(feature_extractor)
    predictions = layers.Dense(class_count, activation='softmax')(hidden_layer)
    model = Model(inputs=input_layer, outputs=predictions)
    model.compile(optimizer ='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
    return model

model = build_model()
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 lambda_6 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 keras_layer_5 (KerasLayer)  (None, 1024)              1026552   
                                                                 
 dense_9 (Dense)             (None, 512)               524800    
                                                                 
 dense_10 (Dense)            (None, 14)                7182      
                                                                 
Total params: 1558534 (5.95 MB)
Trainable params: 531982 (2.03 MB)
Non-trainable params: 1026552 (3.92 MB)
_________________________________________________________________


## 8. Train Model

In [ ]:
%%time
# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=[tensorboard_callback])


### 8.2 Training Result

In [ ]:
# access metrics from training history
print("\ninitial accuracy: {} | latest accuracy: {}".format(history.history["accuracy"][0], history.history["accuracy"][-1]))
print("initial loss: {} | latest loss: {}".format(history.history["loss"][0], history.history["loss"][-1]))

fig, axis = plt.subplots(1, 2, figsize=(10,4)) 
# plot accuracy
axis[0].plot(history.history["accuracy"])
axis[0].set_title("accuracy per epoch")
# plot loss
axis[1].plot(history.history["loss"])
axis[1].set_title("loss per epoch")
plt.show()

## 9. Test Model

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)

print('Test loss:', test_loss, 'Test accuracy:', test_acc)